In [1]:
import numpy as np
import pandas as pd

import tensorflow as tf

from tensorflow import feature_column
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from tensorflow.keras import regularizers
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.callbacks import ReduceLROnPlateau

from imblearn.over_sampling import SMOTE


Tutorial link: https://www.tensorflow.org/tutorials/structured_data/feature_columns

In [3]:
dataframe = pd.read_csv("health.csv")

In [4]:
dataframe.dropna(inplace=True)

In [5]:
dataframe.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 19046 entries, 0 to 19045
Data columns (total 53 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   q66     19046 non-null  float64
 1   q67     19046 non-null  float64
 2   q68     19046 non-null  float64
 3   q69     19046 non-null  float64
 4   q70     19046 non-null  float64
 5   q71     19046 non-null  float64
 6   q72     19046 non-null  float64
 7   q73     19046 non-null  float64
 8   q76     19046 non-null  float64
 9   q78     19046 non-null  float64
 10  q83     19046 non-null  float64
 11  q96     19046 non-null  float64
 12  q97     19046 non-null  float64
 13  q106    19046 non-null  float64
 14  q8      19046 non-null  float64
 15  q9      19046 non-null  float64
 16  q10     19046 non-null  float64
 17  q11     19046 non-null  float64
 18  q12     19046 non-null  float64
 19  q17     19046 non-null  float64
 20  q19     19046 non-null  float64
 21  q26     19046 non-null  float64
 22

In [6]:
dataframe['target'] = dataframe['MB']


In [7]:
dataframe.head()

,q66,q67,q68,q69,q70,q71,q72,q73,q76,q78,...,q79,q80,q81,q82,q84,q85,q87,q92,MB,target
0,0.25,0.000000,0.166667,0.333333,0.0,0.000000,0.0,0.000000,0.000000,0.333333,...,0.00,0.714286,0.0,0.0,0.00,0.666667,0.200804,0.2,0.0,0.0
1,0.75,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.571429,0.333333,...,0.00,0.428571,0.0,0.0,0.50,0.333333,0.500000,0.0,0.0,0.0
2,0.75,0.000000,0.333333,0.166667,0.0,0.166667,0.0,0.166667,1.000000,0.000000,...,0.00,1.000000,0.0,0.0,0.25,1.000000,0.333333,0.0,1.0,1.0
3,0.50,0.333333,0.833333,0.833333,0.0,0.000000,0.0,0.000000,0.285714,0.333333,...,0.00,1.000000,0.0,0.0,0.50,0.500000,0.333333,0.0,1.0,1.0
4,0.50,0.666667,0.333333,0.333333,0.5,0.166667,0.0,0.166667,1.000000,0.000000,...,0.25,0.714286,0.0,0.0,0.50,0.666667,0.500000,0.0,1.0,1.0


In [8]:
dataframe = dataframe.drop(['MB'], axis=1)

In [9]:
train, test = train_test_split(dataframe, test_size=0.2)
train, val = train_test_split(train, test_size=0.125)
print(len(train), 'train examples')
print(len(val), 'validation examples')
print(len(test), 'test examples')

13331 train examples
1905 validation examples
3810 test examples


In [10]:
def df_to_dataset(dataframe, shuffle=True, batch_size=32):
  dataframe = dataframe.copy()
  labels = dataframe.pop('target')
  ds = tf.data.Dataset.from_tensor_slices((dict(dataframe), labels))
  if shuffle:
    ds = ds.shuffle(buffer_size=len(dataframe))
  ds = ds.batch(batch_size)
  return ds

In [11]:
X_train = train.drop('target', axis=1)  
y_train = train['target']  

In [12]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

In [13]:
train_resampled = X_train_resampled.copy()
train_resampled['target'] = y_train_resampled

In [14]:
feature_columns = []

for header in ["q66", "q67", "q68", "q69", "q70", "q71", "q72", "q73", "q76", "q78", "q83", "q96", "q97", "q106", "q8", "q9", "q10", "q11", "q12", "q17",	"q19", "q26", "q27", "q28",	"q29", "q30", "q31", "q35", "q38", "q39", "q46", "q49", "q50", "q51", "q52", "q53", "q54", "q55",	"q58", "q59", "q60", "q61",	"q74", "q75", "q79", "q80", "q81", "q82", "q84", "q85", "q87", "q92"]:
    feature_columns.append(feature_column.numeric_column(header))


Instructions for updating:
Use Keras preprocessing layers instead, either directly or via the `tf.keras.utils.FeatureSpace` utility. Each of `tf.feature_column.*` has a functional equivalent in `tf.keras.layers` for feature preprocessing when training a Keras model.


In [15]:
feature_layer = tf.keras.layers.DenseFeatures(feature_columns)

In [16]:
batch_size = 32
train_ds = df_to_dataset(train_resampled, batch_size=batch_size)
val_ds = df_to_dataset(val, shuffle=False, batch_size=batch_size)
test_ds = df_to_dataset(test, shuffle=False, batch_size=batch_size)

In [17]:
model = tf.keras.Sequential([
    feature_layer,
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dense(128, activation='relu', kernel_regularizer=regularizers.l2(0.001)),
    layers.Dropout(.3),
    layers.Dense(1)
])

In [18]:
from sklearn.utils.class_weight import compute_class_weight

classes = np.unique(train['target'])
class_weights = compute_class_weight(
    class_weight='balanced', 
    classes=classes, 
    y=train['target']
)
class_weight_dict = {i: class_weights[i] for i in range(len(classes))}
print(f"Class weights: {class_weight_dict}")

Class weights: {0: 1.5915711556829035, 1: 0.7290276714426337}


In [19]:
def specificity(y_true, y_pred):
    y_pred = tf.round(tf.nn.sigmoid(y_pred))  
    true_negatives = tf.reduce_sum(tf.cast((y_true == 0) & (y_pred == 0), 'float'))
    false_positives = tf.reduce_sum(tf.cast((y_true == 0) & (y_pred == 1), 'float'))
    
    specificity = true_negatives / (true_negatives + false_positives + tf.keras.backend.epsilon())
    return specificity

In [20]:
def f1_score(y_true, y_pred):
    y_pred = tf.round(tf.nn.sigmoid(y_pred))  
    tp = tf.reduce_sum(tf.cast(y_true * y_pred, 'float'))
    fp = tf.reduce_sum(tf.cast((1 - y_true) * y_pred, 'float'))
    fn = tf.reduce_sum(tf.cast(y_true * (1 - y_pred), 'float'))

    precision = tp / (tp + fp + tf.keras.backend.epsilon())
    recall = tp / (tp + fn + tf.keras.backend.epsilon())
    f1 = 2 * (precision * recall) / (precision + recall + tf.keras.backend.epsilon())
    return f1

In [21]:
model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=[
                  'accuracy',
                  tf.keras.metrics.AUC(name='auc'),
                  specificity,
                  tf.keras.metrics.Recall(name='sensitivity'),
                  f1_score
                      ]
             )

In [22]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

In [23]:
lr_scheduler = ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=3, min_lr=1e-6)


In [24]:
model.fit(
    train_ds,
    validation_data=val_ds,
    epochs=50,
    callbacks=[early_stopping, lr_scheduler]
)

Epoch 1/50


2025-01-11 15:47:46.801016: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 8702 of 18286
2025-01-11 15:47:56.818828: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 10460 of 18286
2025-01-11 15:48:16.818037: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 12672 of 18286
2025-01-11 15:48:36.801745: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 14304 of 18286
2025-01-11 15:48:46.825421: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 15109 of 18286
2025-01-11 15:49:06.794602: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may

134/572 [======>.......................] - ETA: 0s - loss: 0.5223 - accuracy: 0.8263 - auc: 0.8777 - specificity: 0.8551 - sensitivity: 0.7027 - f1_score: 0.8342       

2025-01-11 15:49:44.173068: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


572/572 [==============================] - 129s 1ms/step - loss: 0.3600 - accuracy: 0.8838 - auc: 0.9159 - specificity: 0.9059 - sensitivity: 0.8155 - f1_score: 0.8855 - val_loss: 0.2215 - val_accuracy: 0.9265 - val_auc: 0.9545 - val_specificity: 0.9282 - val_sensitivity: 0.9162 - val_f1_score: 0.9561 - lr: 0.0010
Epoch 2/50


...

572/572 [==============================] - 160s 739us/step - loss: 0.0346 - accuracy: 0.9960 - auc: 0.9984 - specificity: 0.9997 - sensitivity: 0.9919 - f1_score: 0.9981 - val_loss: 0.0397 - val_accuracy: 0.9911 - val_auc: 0.9984 - val_specificity: 0.9927 - val_sensitivity: 0.9894 - val_f1_score: 0.9974 - lr: 1.0000e-06
Epoch 38/50


2025-01-11 17:26:53.927781: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 8163 of 18286
2025-01-11 17:27:03.953000: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 8635 of 18286
2025-01-11 17:27:23.949621: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 11867 of 18286
2025-01-11 17:27:43.946955: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 13398 of 18286
2025-01-11 17:28:03.922626: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may take a while): 14509 of 18286
2025-01-11 17:28:23.917301: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:422] ShuffleDatasetV3:1: Filling up shuffle buffer (this may 

173/572 [========>.....................] - ETA: 0s - loss: 0.0344 - accuracy: 0.9962 - auc: 0.9984 - specificity: 0.9997 - sensitivity: 0.9924 - f1_score: 0.9982    

2025-01-11 17:29:13.742801: I tensorflow/core/kernels/data/shuffle_dataset_op.cc:452] Shuffle buffer filled.


572/572 [==============================] - 150s 833us/step - loss: 0.0346 - accuracy: 0.9967 - auc: 0.9985 - specificity: 0.9992 - sensitivity: 0.9934 - f1_score: 0.9981 - val_loss: 0.0397 - val_accuracy: 0.9911 - val_auc: 0.9984 - val_specificity: 0.9927 - val_sensitivity: 0.9894 - val_f1_score: 0.9969 - lr: 1.0000e-06


In [54]:
model.fit(
    test_ds,
    epochs=38,
)

Epoch 1/38
120/120 [==============================] - 0s 719us/step - loss: 0.0431 - accuracy: 0.9916 - auc: 0.9967 - specificity: 0.9929 - sensitivity: 0.9888 - f1_score: 0.9950
Epoch 2/38
120/120 [==============================] - 0s 696us/step - loss: 0.0428 - accuracy: 0.9924 - auc: 0.9964 - specificity: 0.9930 - sensitivity: 0.9900 - f1_score: 0.9953
Epoch 3/38
120/120 [==============================] - 0s 665us/step - loss: 0.0426 - accuracy: 0.9927 - auc: 0.9972 - specificity: 0.9912 - sensitivity: 0.9904 - f1_score: 0.9957
Epoch 4/38
120/120 [==============================] - 0s 649us/step - loss: 0.0428 - accuracy: 0.9913 - auc: 0.9972 - specificity: 0.9918 - sensitivity: 0.9900 - f1_score: 0.9958
Epoch 5/38
120/120 [==============================] - 0s 666us/step - loss: 0.0429 - accuracy: 0.9927 - auc: 0.9966 - specificity: 0.9896 - sensitivity: 0.9911 - f1_score: 0.9954
Epoch 6/38
120/120 [==============================] - 0s 676us/step - loss: 0.0432 - accuracy: 0.9908 - a

In [56]:
weights = model.get_weights()


In [58]:
first_layer_weights = weights[0]  

weight_df = pd.DataFrame(
    first_layer_weights[:13],  
    columns=[f'Hidden_Node_{i}' for i in range(first_layer_weights.shape[1])], 
    index=X_train.columns[:13]  
)

In [60]:
feature_importance = weight_df.abs().sum(axis=1).sort_values(ascending=False)

print(feature_importance)

q67    4.221607
q97    4.126722
q72    3.920302
q73    3.880244
q76    3.876490
q66    3.763830
q68    3.714168
q69    3.710713
q96    3.689796
q71    3.449429
q83    3.378315
q70    2.952023
q78    2.877127
dtype: float32


In [63]:
weight_mb = pd.DataFrame(first_layer_weights, columns=[f'Hidden_Node_{i}' for i in range(first_layer_weights.shape[1])], index=X_train.columns)

mb_importance = weight_mb.abs().sum(axis=1).sort_values(ascending=False)
print(mb_importance)

q60     5.299533
q87     4.927851
q35     4.795099
q61     4.793812
q81     4.694726
q55     4.402696
q92     4.369410
q54     4.304658
q80     4.251403
q67     4.221607
q97     4.126722
q58     4.104058
q46     4.074771
q38     4.025519
q52     4.022034
q82     3.999923
q28     3.943447
q30     3.926598
q72     3.920302
q73     3.880244
q9      3.878940
q76     3.876490
q53     3.869617
q51     3.766945
q66     3.763830
q31     3.723319
q68     3.714168
q39     3.713171
q69     3.710713
q96     3.689796
q74     3.685955
q59     3.660229
q50     3.649377
q79     3.641234
q75     3.569131
q10     3.505089
q71     3.449429
q84     3.413808
q49     3.405173
q83     3.378315
q70     2.952023
q78     2.877127
q12     2.761600
q29     2.753660
q85     2.700336
q106    2.218318
q17     2.071753
q26     1.975042
q8      1.413758
q11     1.407523
q19     0.678832
q27     0.471366
dtype: float32
